In [1]:
import advent
advent.scrape(2016, 25)
data = advent.get_lines(25)

In [51]:
# Copied from day 23, removed tgl instruction and added out instruction

def lookup(registers, v):
    if v in ['a', 'b', 'c', 'd']: return registers[v]
    return int(v)

def run_program(registers: dict[str, int], data: list[str], maxout=2, debugins=''):
    instruction_pointer = 0
    output = []
    while instruction_pointer < len(data):
        ins = data[instruction_pointer]
        if ins == debugins:
            print(registers, output)
        if ins.startswith('cpy'):
            _, v, r = ins.split()
            registers[r] = lookup(registers, v)
        elif ins.startswith('inc'):
            _, r = ins.split()
            registers[r] += 1
        elif ins.startswith('dec'):
            _, r = ins.split()
            registers[r] -= 1
        elif ins.startswith('jnz'):
            _, r, v = ins.split()
            if lookup(registers, r) != 0:
                instruction_pointer += (lookup(registers, v) - 1)
        elif ins.startswith('out'):
            _, r = ins.split()
            output.append(registers[r])
        instruction_pointer += 1
        if len(output) == maxout:
            return output
    raise ValueError()

print(run_program({'a': 1, 'b': 0, 'c': 0, 'd': 0}, data)) # well that doesn't look like [0, 1]
print(run_program({'a': 100, 'b': 0, 'c': 0, 'd': 0}, data)) # well that doesn't look like [0, 1]
print(run_program({'a': 200, 'b': 0, 'c': 0, 'd': 0}, data)) # well that doesn't look like [0, 1]

[0, 0]
[1, 1]
[1, 1]


In [52]:
# Try brute force
#for i in range(1000):
#    output = run_program({'a': 1, 'b': 0, 'c': 0, 'd': 0}, data)
#    if all(o == 1 for o in output[1::2]) and all(o == 0 for o in output[::2]):
#        print(i)

In [53]:
# Brute force didn't really work, let's try something smarter...
_ = run_program({'a': 100, 'b': 0, 'c': 0, 'd': 0}, data, debugins='cpy 0 a')
print('--')
_ = run_program({'a': 200, 'b': 0, 'c': 0, 'd': 0}, data, debugins='cpy 0 a')

# From this, it becomes clear that at the cpy 0 a step, when we first reach it:
# d = 2555 + input (incidentally, 2555 = 365*7, which explains where the 2555 comes from, looking at the program)


{'a': 2655, 'b': 2655, 'c': 0, 'd': 2655} []
{'a': 1327, 'b': 1327, 'c': 0, 'd': 2655} [1]
--
{'a': 2755, 'b': 2755, 'c': 0, 'd': 2755} []
{'a': 1377, 'b': 1377, 'c': 0, 'd': 2755} [1]


In [61]:
_ = run_program({'a': 100, 'b': 0, 'c': 0, 'd': 0}, data, debugins='cpy 2 b')
print('--')
_ = run_program({'a': 207, 'b': 0, 'c': 0, 'd': 0}, data, debugins='cpy 2 b')

# At this stage, a is 2655//2, c is 2 if a is even, 1 if a is odd
# The rest of the program:

# cpy 2 b  - b = 2
# jnz c 2  - initially always triggers since c = 1 or 2
# jnz 1 4  - skips to jnz 0 0 if c is 0
# dec b    - 
# dec c    - 
# jnz 1 -4 - skips back to jnz c 2
# jnz 0 0  - -
# out b    - b being 0 here, meaning c must have been 2 to begin with
#          - b being 1 here, meaning c must have been 1 to begin with

# Conclusion: we want c to be 2, then 1, then 2 again.
# meaning we want a to be even, then odd, then even, ...
# meaning that:
# let d = 2555 + input
# d // 2 must be even -> d in binary is xxxxx0x
# (d // 2) // 2 must be odd -> d in binary is xxxx10x

# 2555 in binary is 100111111011
# we want it to be 1010101010100
print(0b1010101010100 - 0b100111111011)
assert format(2555 + 2905, 'b') == '1010101010100'

{'a': 1327, 'b': 0, 'c': 1, 'd': 2655} []
{'a': 663, 'b': 0, 'c': 1, 'd': 2655} [1]
--
{'a': 1381, 'b': 0, 'c': 2, 'd': 2762} []
{'a': 690, 'b': 0, 'c': 1, 'd': 2762} [0]
2905


In [60]:
print(run_program({'a': 2905, 'b': 0, 'c': 0, 'd': 0}, data, maxout=20))
# Ok this is really annoying... apparently we dont want it to end in 00 but just in 0
# 2555 in binary is 100111111011
# we want it to be  101010101010
print(0b101010101010 - 0b100111111011)
assert format(2555 + 175, 'b') == '101010101010'

[0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]
175


In [ ]:
print(run_program({'a': 175, 'b': 0, 'c': 0, 'd': 0}, data, maxout=20))
# So the answer is 175
# I'm not sure where I made the off by one error, but oh well, this works :D

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
